In [ ]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

Quantum GANs {#quantum_gans}
============

::: {.meta}
:property=\"og:description\": Explore quantum GANs to generate
hand-written digits of zero :property=\"og:image\":
<https://pennylane.ai/qml/_images/patch.jpeg>
:::

::: {.related}
tutorial\_QGAN Quantum generative adversarial networks with Cirq +
TensorFlow
:::


*Author: James Ellis --- Posted: 01 February 2022. Last updated: 27
January 2022.*

In this tutorial, we will explore quantum GANs to generate hand-written
digits of zero. We will first cover the theory of the classical case,
then extend to a quantum method recently proposed in the literature. If
you have no experience with GANs, particularly in PyTorch, you might
find [PyTorch\'s
tutorial](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html)
useful since it serves as the foundation for what is to follow.


Generative Adversarial Networks (GANs)
======================================


The goal of generative adversarial networks (GANs) is to generate data
that resembles the original data used in training. To achieve this, we
train two neural networks simulatenously: a generator and a
discriminator. The job of the generator is to create fake data which
imitates the real training dataset. On the otherhand, the discriminator
acts like a detective trying to discern real from fake data. During the
training process, both players iteratively improve with one another. By
the end, the generator should hopefully generate new data very similar
to the training dataset.

Specifically, the training dataset represents samples drawn from some
unknown data distribution $P_{data}$, and the generator has the job of
trying to capture this distribution. The generator, $G$, starts from
some initial latent distribution, $P_z$, and maps it to $P_g = G(P_z)$.
The best solution would be for $P_g = P_{data}$. However, this point is
rarely achieved in practice apart from in the most simple tasks.

Both the discriminator, $D$, and generator, $G$, play in a 2-player
minimax game. The discriminator tries to maximise the probability of
discerning real from fake data, while the generator tries to minimise
the same probability. The value function for the game is summarised by,

$$\begin{aligned}
\begin{align}
\min_G \max_D V(D,G) &= \mathbb{E}_{\boldsymbol{x}\sim p_{data}}[\log D(\boldsymbol{x})] \\
    & ~~ + \mathbb{E}_{\boldsymbol{z}\sim p_{\boldsymbol{z}}}[\log(1 - D(G(\boldsymbol{z}))]
\end{align}
\end{aligned}$$

-   $\boldsymbol{x}$: real data sample
-   $\boldsymbol{z}$: latent vector
-   $D(\boldsymbol{x})$: probability of the discriminator classifying
    real data as real
-   $G(\boldsymbol{z})$: fake data
-   $D(G(\boldsymbol{z}))$: probability of discriminator classifying
    fake data as real

In practice, the two networks are trained iteratively, each with a
separate loss function to be minimised,

$$L_D = -[y \cdot \log(D(x)) + (1-y)\cdot \log(1-D(G(z)))]$$

$$L_G = [(1-y) \cdot \log(1-D(G(z)))]$$

where $y$ is a binary label for real ($y=1$) or fake ($y=0$) data. In
practice, generator training is shown to be more stable when made to
maximise $\log(D(G(z)))$ instead of minimising $\log(1-D(G(z)))$. Hence,
the generator loss function to be minimised becomes,

$$L_G = -[(1-y) \cdot \log(D(G(z)))]$$


Quantum GANs: The Patch Method
==============================


In this tutorial, we re-create one of the quantum GAN methods presented
by Huang et al.: the patch method. This method uses several quantum
generators, with each sub-generator, $G^{(i)}$, responsible for
constructing a small patch of the final image. The final image is
contructed by concatenting all of the patches together as shown below.

![](../demonstrations/quantum_gans/patch.jpeg){.align-center
width="90.0%"}

The main advantage of this method is that it is particulary suited to
situations where the number of available qubits are limited. The same
quantum device can be used for each sub-generator in an iterative
fashion, or execution of the generators can be parallelised across
multiple devices.

::: {.note}
::: {.title}
Note
:::

In this tutorial, parenthesised superscripts are used to denote
individual objects as part of a collection.
:::


Module Imports
==============


In [ ]:
# Library imports








# Pytorch imports







# Set the random seed for reproducibility

Data
====


As mentioned in the introduction, we will use a [small
dataset](https://archive.ics.uci.edu/ml/datasets/optical+recognition+of+handwritten+digits)
of handwritten zeros. First, we need to create a custom dataloader for
this dataset.


Next we define some variables and create the dataloader instance.


Let\'s visualize some of the data.


Implementing the Discriminator
==============================


For the discriminator, we use a fully connected neural network with two
hidden layers. A single output is sufficient to represent the
probability of an input being classified as real.


Implementing the Generator
==========================


Each sub-generator, $G^{(i)}$, shares the same circuit architecture as
shown below. The overall quantum generator consists of $N_G$
sub-generators, each consisting of $N$ qubits. The process from latent
vector input to image output can be split into four distinct sections:
state embedding, parameterisation, non-linear transformation, and
post-processing. Each of the following sections below refer to a single
iteration of the training process to simplify the discussion.

![](../demonstrations/quantum_gans/qcircuit.jpeg){.align-center
width="90.0%"}

**1) State Embedding**

A latent vector, $\boldsymbol{z}\in\mathbb{R}^N$, is sampled from a
uniform distribution in the interval $[0,\pi/2)$. All sub-generators
receive the same latent vector which is then embedded using RY gates.

**2) Parameterised Layers**

The parameterised layer consists of parameterised RY gates followed by
control Z gates. This layer is repeated $D$ times in total.

**3) Non-Linear Transform**

Quantum gates in the circuit model are unitary which, by definition,
linearly transform the quantum state. A linear mapping between the
latent and generator distribution would suffice for only the most simple
generative tasks, hence we need non-linear transformations. We will use
ancillary qubits to help.

For a given sub-generator, the pre-measurement quantum state is given
by,

$$|\Psi(z)\rangle = U_{G}(\theta)|\boldsymbol{z}\rangle$$

where $U_{G}(\theta)$ represents the overall unitary of the
parameterised layers. Let us inspect the state when we take a partial
measurment, $\Pi$, and trace out the ancillary subsystem, $\mathcal{A}$,

$$\rho(\boldsymbol{z}) = \frac{\text{Tr}_{\mathcal{A}}(\Pi \otimes \mathbb{I} |\Psi(z)\rangle \langle \Psi(\boldsymbol{z})|) }{\text{Tr}(\Pi \otimes \mathbb{I} |\Psi(\boldsymbol{z})\rangle \langle \Psi(\boldsymbol{z})|))} = \frac{\text{Tr}_{\mathcal{A}}(\Pi \otimes \mathbb{I} |\Psi(\boldsymbol{z})\rangle \langle \Psi(\boldsymbol{z})|) }{\langle \Psi(\boldsymbol{z})| \Pi \otimes \mathbb{I} |\Psi(\boldsymbol{z})\rangle}$$

The post-measurement state, $\rho(\boldsymbol{z})$, is dependent on
$\boldsymbol{z}$ in both the numerator and denominator. This means the
state has been non-linearly transformed! For this tutorial,
$\Pi = (|0\rangle \langle0|)^{\otimes N_A}$, where $N_A$ is the number
of ancillary qubits in the system.

With the remaining data qubits, we measure the probability of
$\rho(\boldsymbol{z})$ in each computational basis state, $P(j)$, to
obtain the sub-generator output, $\boldsymbol{g}^{(i)}$,

$$\boldsymbol{g}^{(i)} = [P(0), P(1), ... ,P(2^{N-N_A} - 1)]$$

**4) Post Processing**

Due to the normalisation constraint of the measurment, all elements in
$\boldsymbol{g}^{(i)}$ must sum to one. This is a problem if we are to
use $\boldsymbol{g}^{(i)}$ as the pixel intensity values for our patch.
For example, imagine a hypothetical situation where a patch of full
intensity pixels was the target. The best patch a sub-generator could
produce would be a patch of pixels all at a magnitude of
$\frac{1}{2^{N-N_A}}$. To alleviate this constraint, we apply a
post-processing technique to each patch,

$$\boldsymbol{\tilde{x}^{(i)}} = \frac{\boldsymbol{g}^{(i)}}{\max_{k}\boldsymbol{g}_k^{(i)}}$$

Therefore, the final image, $\boldsymbol{\tilde{x}}$, is given by

$$\boldsymbol{\tilde{x}} = [\boldsymbol{\tilde{x}^{(1)}}, ... ,\boldsymbol{\tilde{x}^{(N_G)}}]$$


In [ ]:
# Quantum variables

Now we define the quantum device we want to use, along with any
available CUDA GPUs (if available).


In [ ]:
# Quantum simulator

# Enable CUDA device if available

Next, we define the quantum circuit and measurement process described
above.


In [ ]:
# For further info on how the non-linear transform is implemented in Pennylane
# https://discuss.pennylane.ai/t/ancillary-subsystem-measurement-then-trace-out/1532

Now we create a quantum generator class to use during training.


Training
========


Let\'s define learning rates and number of iterations for the training
process.


Now putting everything together and executing the training process.


In [ ]:
# Binary cross entropy


# Optimisers






# Fixed noise allows us to visually track the generated images throughout training


# Iteration counter


# Collect images for plotting later

::: {.note}
::: {.title}
Note
:::

You may have noticed `errG = criterion(outD_fake, real_labels)` and
wondered why we don't use `fake_labels` instead of `real_labels`.
However, this is simply a trick to be able to use the same `criterion`
function for both the generator and discriminator. Using `real_labels`
forces the generator loss function to use the $\log(D(G(z))$ term
instead of the $\log(1 - D(G(z))$ term of the binary cross entropy loss
function.
:::


Finally, we plot how the generated images evolved throughout training.


Acknowledgements
================

Many thanks to Karolis Špukas who I co-developed much of the code with.
I also extend my thanks to Dr. Yuxuan Du for answering my questions
regarding his paper. I am also indebited to the Pennylane community for
their help over the past few years.

References
==========

About the author
================
